In [17]:
import time, random, sys
import numpy as np
from math import sin, cos, sqrt, atan2, radians
from opensky_api import OpenSkyApi
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from kf.kf import KF
#import smopy
#from scipy.misc import imresize

In [18]:
%matplotlib

Using matplotlib backend: TkAgg


In [19]:
MIN_NUMBER_OF_UPDATES = 4
"""
# Prague, (Morina - Sojovice)
lat_min, lon_min = 49.951508, 14.212285
lat_max, lon_max = 50.221933, 14.763462
lat_center, lon_center = 50.101770, 14.263117
lat_min, lon_min = lat_center - 0.2, lon_center - 0.2
lat_max, lon_max = lat_center + 0.2, lon_center + 0.2
airport_name = "Vaclav Havel's Airport"
# Vaclav Havel's Airport
lat_min, lon_min = 50.074975, 14.199871
lat_max, lon_max= 50.130103, 14.324053
lat_center, lon_center = 50.101770, 14.263117
airport_name = "Vaclav Havel's Airport"
"""
# London Heathrow Airport (Sunningdale - Wembley)
lat_min, lon_min = 51.392666, -0.633652
lat_max, lon_max = 51.550291, -0.301164
lat_center, lon_center = 51.467612, -0.453609
airport_name = "London Heathrow Airport"
"""
# Chicago O'Hare International Airport (Wheaton - somewhere in the Michigan lake)
lat_min, lon_min = 41.868163, -88.097664
lat_max, lon_max = 42.163555, -87.601343
lat_center, lon_center = 41.980317, -87.912309
airport_name = "Chicago O'Hare International Airport"

# Chicago O'Hare International Airport (Addison - Park Ridge)
lat_min, lon_min = 41.931889, -87.987572
lat_max, lon_max = 42.011809, -87.834858
lat_center, lon_center = 41.980317, -87.912309
airport_name = "Chicago O'Hare International Airport"
#"""
dlon = (lon_max - lon_min)
dlat = (lat_max - lat_min)
ratio = dlon / dlat
api = OpenSkyApi()

In [20]:
def lat_lon_to_dist(lat_min, lon_min, lat_max, lon_max):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat_min)
    lon1 = radians(lon_min)
    lat2 = radians(lat_max)
    lon2 = radians(lon_max)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c * 1000

In [21]:
def init_kalman(delta_time, q, r):
    dt = delta_time
    A = np.array([
        [1, 0, 0, dt, 0, 0],
        [0, 1, 0, 0, dt, 0],
        [0, 0, 1, 0, 0, dt],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]
    ])
    B = None
    H = np.array([
        [1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0]
    ])
    R = r**2 * np.eye(3)
    Q = q * np.diag([dt, dt, dt, dt, dt, dt])
    kf = KF(A=A, B=B, H=H, R=R, Q=Q)
    return kf

In [22]:
def update_data(s, data, kfs):
    data.append({})
    # Process data.
    for state in s.states:
        # Filter unnamed flights and flights with None altitude.
        if len(state.callsign) < 1: continue
        if state.on_ground is False and state.baro_altitude is None: continue
        if state.on_ground: continue
        
        #lat, lon = mp.to_pixels(state.latitude, state.longitude)
        #lat, lon = state.latitude, state.longitude
        lon = lat_lon_to_dist(lat_min, lon_min, lat_min, state.longitude)
        lat = lat_lon_to_dist(lat_min, lon_min, state.latitude, lon_min)
        
        if state.callsign not in kfs:
            # If the flight is new, create new Kalman filter for it.
            kf = init_kalman(delta_time=1., q=2., r=.5)
            kfs[state.callsign] = [kf, 0]
        kf = kfs[state.callsign][0]
        
        if state.on_ground:
            data[-1][state.callsign] = (lon, lat, 0)
            # TODO ?
            kf.predict()
            kf.update(np.array([lon, lat, 0]))
            kfs[state.callsign][1] += 1
        else:
            data[-1][state.callsign] = (lon, lat, state.baro_altitude)
            # TODO ?
            kf.predict()
            kf.update(np.array([lon, lat, state.baro_altitude]))
            kfs[state.callsign][1] += 1

In [23]:
def predict_kalman(data, kfs):
    data.append({})
    for flight in data[-2]:
        kf, number_of_updates = kfs[flight]
        kf.predict()
        if number_of_updates < MIN_NUMBER_OF_UPDATES:
            continue
        else:
            data[-1][flight] = (kf.x[0], kf.x[1], kf.x[2])

In [ ]:
################################ Initialize variables ##################################
s, line = None, None
lst = []
anns, d, color_dict, kfs, prevs = {}, {}, {}, {}, {}
i, c, prev_time = 0, 8, -1
kf = False

################################ Initialize figure #####################################
fig = plt.figure(figsize=(c * ratio, c))
ax = fig.add_subplot(111, projection='3d')

#mp = smopy.Map((lat_min, lon_min, lat_max, lon_max))
#img = mp.to_numpy() / 255.0
#x, y = np.ogrid[0:img.shape[0], 0:img.shape[1]]
#ax.plot_surface(x, y, np.zeros(y.shape), rstride=5, cstride=5, facecolors=img)

xlim = lat_lon_to_dist(lat_min, lon_min, lat_min, lon_max)
ylim = lat_lon_to_dist(lat_min, lon_min, lat_max, lon_min)
ax.set_xlim(0, xlim)
ax.set_ylim(0, ylim)
ax.set_zlim(0, 2500)

#aa, bb = mp.to_pixels(lat_center, lon_center)
#aa, bb = lat_center, lon_center
bb = lat_lon_to_dist(lat_min, lon_min, lat_min, lon_center)
aa = lat_lon_to_dist(lat_min, lon_min, lat_center, lon_min)
ax.plot([bb], [aa], [0], 'o', c='green', markersize=16, label = airport_name)
plt.legend()

################################ Tracking airplanes #####################################
i = 0
s2 = api.get_states(bbox = (lat_min, lat_max, lon_min, lon_max))
start = s2.time
start_real = int(time.time())
cur = start
while True:
    if cur - start < i:
        # Wait until at least i seconds pass since the start.
        t = ax.text2D(0.05, 0.95, str(i), transform=ax.transAxes, fontsize=14, verticalalignment='top')
        plt.pause(0.01)
        t.remove()
        cur = start + (int(time.time()) - start_real)
        continue
    i += 1
    if i == 1:
        # i == 1, special case of the first iteration.
        update_data(s2, lst, kfs)
        prev_time = s2.time
    else:
        s2 = api.get_states(bbox = (lat_min, lat_max, lon_min, lon_max))
        if s2 is None:
            # Predict positions using 3D Kalman filter.
            predict_kalman(lst, kfs)
        elif s2.time == prev_time:
            # Predict positions using 3D Kalman filter.
            predict_kalman(lst, kfs)
        else:
            # Update our data with obtained real data.
            update_data(s2, lst, kfs)
            prev_time = s2.time
    
    if len(lst[-1]) == 0:
        continue
    
    # Load data for annotations.
    anns_info = {}
    for flight in lst[-1]:
        if flight not in color_dict:
            color_dict[flight] = np.random.rand(3,)
        lon, lat, alt = lst[-1][flight]
        anns_info[flight] = (lon, lat, alt)
    # Delete old annotations from the figure.
    for flight in lst[-1]:
        if flight not in anns:
            continue
        anns[flight].remove()
        del anns[flight]

    # Plot figure.
    if len(lst) == 1:
        # Special case of the first iteration.
        for flight in lst[-1]:
            line, = ax.plot([lst[-1][flight][0]], [lst[-1][flight][1]], [lst[-1][flight][2]], 'x', c='black')
            prevs[flight] = lst[-1][flight]
        for flight in anns_info:
            item = anns_info[flight]
            ann = ax.text(item[0], item[1], item[2], flight)
            anns[flight] = ann
    else:
        # All other iterations.
        for flight in lst[-1]:
            if flight in prevs:
                prev_flight = prevs[flight]
                ax.plot([prev_flight[0], lst[-1][flight][0]], [prev_flight[1], lst[-1][flight][1]], [prev_flight[2], lst[-1][flight][2]], '-x', c=color_dict[flight])
                prevs[flight] = lst[-1][flight]
            else:
                line, = ax.plot([lst[-1][flight][0]], [lst[-1][flight][1]], [lst[-1][flight][2]], 'x', c='black')
                prevs[flight] = lst[-1][flight]
        for flight in anns_info:
            item = anns_info[flight]
            ann = ax.text(item[0], item[1], item[2], flight)
            anns[flight] = ann
        #plt.draw()
    #plt.legend()